# MOVIE ANALYSIS

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/05-loading-data.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/05-loading-data.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/05-loading-data.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

### Start EVA Server
We are reusing the start server notebook for launching the EVA server

In [ ]:
!wget -nc "https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/00-start-eva-server.ipynb"
%run 00-start-eva-server.ipynb
cursor = connect_to_server()

In [7]:
import sys
sys.path.insert(0,'..')
from eva.server.db_api import connect

import nest_asyncio
nest_asyncio.apply()
connection = connect(host = '0.0.0.0', port = 5432) # hostname, port of the server where EVADB is running

cursor = connection.cursor()

### Video Files
 - Download dataset

In [2]:
# sample dataset of 2 videos
!wget -nc https://www.dropbox.com/s/jnkil8py0ng5za5/bddtest.zip

# unzip
!unzip -n bddtest.zip

File ‘bddtest.zip’ already there; not retrieving.

Archive:  bddtest.zip


- Load the 2 videos

In [3]:
cursor.execute('LOAD FILE "./bddtest/videos/bddtest_00a2e3ca5c856cde.mp4" INTO bddtest_1;')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD FILE "./bddtest/videos/bddtest_00a395fed60c0b47.mp4" INTO bddtest_2;')
response = cursor.fetch_all()
print(response)

@status: 0
@batch: Batch Object:
@dataframe:                                                    0
0  Video successfully added at location: bddtest/...
@batch_size: 1
@identifier_column: None
@query_time: 0.035945682000601664
@status: 0
@batch: Batch Object:
@dataframe:                                                    0
0  Video successfully added at location: bddtest/...
@batch_size: 1
@identifier_column: None
@query_time: 0.02204383199568838


- Create a table to store the metadata information 

In [4]:
create_table_query = f""" 

    CREATE TABLE IF NOT EXISTS bddtestmeta(
        id INTEGER UNIQUE,
        frame_id INTEGER,
        video_id INTEGER,
        dataset_name TEXT(30),
        label TEXT(30),
        bbox NDARRAY FLOAT32(4),
        object_id INTEGER
    );
    
    """

cursor.execute(create_table_query)
response = cursor.fetch_all()
print(response)

@status: 0
@batch: Batch Object:
@dataframe: Empty DataFrame
Columns: []
Index: []
@batch_size: 0
@identifier_column: None
@query_time: 0.026316527000744827


- Load the 2 corresponding CSV files

In [ ]:
cursor.execute("LOAD FILE 'bddtest/info/bddtest_00a2e3ca5c856cde.csv' INTO bddtestmeta WITH FORMAT CSV;")
response = cursor.fetch_all()
print(response)

cursor.execute("LOAD FILE 'bddtest/info/bddtest_00a395fed60c0b47.csv' INTO bddtestmeta WITH FORMAT CSV;")
response = cursor.fetch_all()
print(response)

## Query from the dataset
- Run a test query on the loaded video

In [8]:
cursor.execute("""SELECT id, FastRCNNObjectDetector(data) 
                  FROM bddtest_1 
                  WHERE id < 20;""")
response = cursor.fetch_all()
print(response)

@status: 0
@batch: Batch Object:
@dataframe:     bddtest_1.id                      fastrcnnobjectdetector.labels  \
0              0  [stop sign, car, stop sign, car, car, car, car...   
1              1  [stop sign, car, car, stop sign, car, car, car...   
2              2  [stop sign, car, car, stop sign, car, car, car...   
3              3  [stop sign, car, car, stop sign, car, car, car...   
4              4  [stop sign, car, car, car, car, car, stop sign...   
5              5  [stop sign, car, car, car, car, stop sign, car...   
6              6  [stop sign, car, car, car, car, stop sign, car...   
7              7  [stop sign, car, car, car, car, stop sign, car...   
8              8  [stop sign, car, car, car, car, stop sign, car...   
9              9  [stop sign, car, car, car, car, car, stop sign...   
10            10  [stop sign, car, car, car, car, car, stop sign...   
11            11  [stop sign, car, car, car, car, car, stop sign...   
12            12  [stop sign, ca

- Run a test query on the loaded metadata (CSV)

In [9]:
cursor.execute("""SELECT id, frame_id, video_id 
                  FROM bddtestmeta;""")
response = cursor.fetch_all()
print(response)

@status: 0
@batch: Batch Object:
@dataframe:        bddtestmeta.id  bddtestmeta.frame_id  bddtestmeta.video_id
0                9881                   583                     4
1                9882                   583                     4
2                9883                   589                     4
3                9884                   589                     4
4                9885                   589                     4
...               ...                   ...                   ...
17000            9876                   583                     4
17001            9877                   583                     4
17002            9878                   583                     4
17003            9879                   583                     4
17004            9880                   583                     4

[17005 rows x 3 columns]
@batch_size: 17005
@identifier_column: None
@query_time: 2.022062427000492
